# Script to assess the success of the aircraft cameras for the most recent projects

In [23]:
import xarray as xr
import numpy as np
import os
import datetime
raw_data_dir = os.getenv('RAW_DATA_DIR')
data_dir =os.getenv('DATA_DIR')
today= datetime.datetime.today().strftime('%Y%m%d')
output_file = f'camera_percentages_{today}.txt'
import subprocess

In [16]:
project = ['CAESAR','WECAN',  'ACCLIP', 'TI3GER','OTREC']
cameras = {'CAESAR':['FWD','DWN'], 'WECAN':['FWD','DOWN'], 'OTREC':['forward','left','right','down'] , 'ACCLIP':['FWD','RIGHT','LEFT'], 'TI3GER':['FWD','LEFT','RIGHT','DOWN'] }
camera_directory = {'CAESAR':'camera_images/hourly_tar', 'WECAN':'CAMERA', 'OTREC':'camera_images', 'ACCLIP':'camera_images', 'TI3GER':'camera_images'}

In [3]:
def sum_hourly(cam_dict):
    hourly_sum = {}
    for file in cam_dict:
        key = file[:4].lower()
        if key in hourly_sum:
            hourly_sum[key] += cam_dict[file]
        else:
            hourly_sum[key] = cam_dict[file]
    return hourly_sum

In [13]:

def get_times(project):
    nc_dir = os.path.join(data_dir, project)
    time_dict = {}
    for file in os.listdir(nc_dir):
        if len(file) != (len(project)+7):
            continue
        if file.endswith(".nc"):
            flight_number = file.split('.')[0][len(project):len(project)+4]  # Extract flight number from filename
            file_path = os.path.join(nc_dir, file)
            result = subprocess.run(['ncdump', '-h', file_path], stdout=subprocess.PIPE, text=True)
            time_line = next((line for line in result.stdout.split('\n') if 'Time =' in line), None)
            if time_line:
                try:
                    time_value = int(time_line.split('=')[1].strip().strip(';'))
                except ValueError:
                    time_value =int(time_line.split("(")[1].split('currently')[0].strip())
                time_dict[flight_number] = time_value
    return time_dict

In [17]:
def get_camera_data(project):
    camera_dir = os.path.join(raw_data_dir, project, camera_directory[project])
    camera = cameras[project]
    cam_dict = {cam: {} for cam in camera}
    if project == 'OTREC':
        for file in os.listdir(camera_dir):
            # Extract the flight number (last four characters of the directory name)
            flight_number = file[-4:].lower()
            for cam in camera:
                sub_dir = os.path.join(camera_dir,file, cam)
                if os.path.exists(sub_dir):
                    # Count the number of files in the subdirectory
                    file_count = len([file for file in os.listdir(sub_dir) if os.path.isfile(os.path.join(sub_dir, file))])
                    cam_dict[cam][flight_number] = file_count
    else:
        for dir in camera:
            for file in os.listdir(camera_dir):
                if file.endswith(".tar.dir"):
                    if dir in file:
                        with open(os.path.join(camera_dir, file), 'r') as f:
                            cam_dict[dir][file] = sum(1 for line in f)

    # Print the data for each camera
    for cam in camera:
        print(f"Data for {cam}:")
        print(cam_dict[cam])
    return cam_dict

In [19]:
def get_percentages(cam_dict, time_dict,project):
    camera = cameras[project]
    sums = {}
    for dir in camera:
        sums[dir] = sum_hourly(cam_dict[dir])


    percentage_dir ={}
    for dir in sums:
        percentage_dir[dir] = {}
        for flight in time_dict:
        
            if flight in sums[dir]:
                percentage_dir[dir][flight] = round(sums[dir][flight] / time_dict[flight] * 100,2)
            else:
                percentage_dir[dir][flight] = 0
    return percentage_dir


In [28]:
def write_output(percentage_dir, project):
    with open(output_file, 'a') as f:
        f.write('##############################################\n')
        f.write(f"Project: {project}\n") 
        f.write('##############################################\n')   
        for dir in percentage_dir:
            output = f"{project}, {dir}, {percentage_dir[dir]}"
            print(output)
            f.write(output + '\n')
        f.write('\n')

In [29]:
for proj in project:
    time_dict = get_times(proj)
    cam_dict = get_camera_data(proj)
    percentage_dir = get_percentages(cam_dict, time_dict, proj)
    write_output(percentage_dir, proj)

Data for FWD:
{'RF04.FWD.240305-050000_055959.tar.dir': 436, 'RF04.FWD.240305-060000_065959.tar.dir': 3170, 'RF04.FWD.240305-070000_075959.tar.dir': 3582, 'RF04.FWD.240305-080000_085959.tar.dir': 3581, 'RF04.FWD.240305-090000_095959.tar.dir': 3581, 'RF04.FWD.240305-100000_105959.tar.dir': 3572, 'RF04.FWD.240305-110000_115959.tar.dir': 3555, 'RF04.FWD.240305-120000_125959.tar.dir': 2346, 'RF05.FWD.240311-050000_055959.tar.dir': 5, 'RF05.FWD.240311-060000_065959.tar.dir': 190, 'RF05.FWD.240311-070000_075959.tar.dir': 3579, 'RF05.FWD.240311-080000_085959.tar.dir': 3583, 'RF05.FWD.240311-090000_095959.tar.dir': 3582, 'RF05.FWD.240311-100000_105959.tar.dir': 3582, 'RF05.FWD.240311-110000_115959.tar.dir': 3578, 'RF05.FWD.240311-120000_125959.tar.dir': 3074, 'RF06.FWD.240312-040000_045959.tar.dir': 4, 'RF06.FWD.240312-050000_055959.tar.dir': 0, 'RF06.FWD.240312-060000_065959.tar.dir': 3125, 'RF06.FWD.240312-070000_075959.tar.dir': 3582, 'RF06.FWD.240312-080000_085959.tar.dir': 3583, 'RF06.FWD

In [ ]:
""" Code to count FF04 but we don't have the netcdf file because it's not shared publicly
down_count = 0
forward_count = 0

with open(os.path.join(raw_data_dir, project, 'camera_images/flight_number_ff04.tar.dir'), 'r') as f:
    for line in f:
        if 'down' in line:
            down_count += 1
        if 'forward' in line:
            forward_count += 1

CAESAR_fwd['ff04'] = forward_count
CAESAR_dwn['ff04'] = down_count

caesar_fwd_percentages = {}
caesar_dwn_percentages = {} """